#Imports

In [1]:
#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys
# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/TER'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/TER


'/content/gdrive/My Drive/Colab Notebooks/TER'

#Classification

In [3]:
import pandas as pd
V1 = pd.read_excel("ISAC_V1.xlsx", engine="openpyxl")
V1.head()

,Unnamed: 0,Chip_Code,Chip_Type,Age,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,...,Min_Animaux,Moyenne_Alimentaires,Max_Alimentaires,Min_Alimentaires,Moyenne_Venins,Max_Venins,Min_Venins,Moyenne_Glycanes,Max_Glycanes,Min_Glycanes
0,XPW0007,XPW0007,ISAC_V1,9,3,32,11,0,0,0,...,0,0.000000,0.00,0,0.1025,0.27,0.0,0.000000,0.00,0.0
1,XPW0011,XPW0011,ISAC_V1,25,5,31,11,1,0,0,...,0,1.505682,36.15,0,0.0000,0.00,0.0,2.278333,10.67,0.0
2,XPW0013,XPW0013,ISAC_V1,59,5,82,11,0,0,0,...,0,0.014091,0.25,0,0.0000,0.00,0.0,0.000000,0.00,0.0
3,XPW0017,XPW0017,ISAC_V1,49,5,33,10,1,0,0,...,0,0.006364,0.21,0,0.0000,0.00,0.0,0.011667,0.07,0.0
4,XPW0018,XPW0018,ISAC_V1,9,5,33,10,1,0,0,...,0,3.612045,36.38,0,0.0175,0.07,0.0,2.791667,16.22,0.0


In [4]:
target_1 = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Food_Allergy_Other",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",
]

extra_columns = [
    "Chip_Type",
    "Chip_Code",
    "French_Region",
    "French_Residence_Department"
    ]

extra = ['History_of_food_anaphylaxis','First_degree_family_history_of_atopy', 'History_of_atopic_dematitis', 'History_of_mast_cell_disorders',
         'History_of_hymenoptera_venom_anaphylaxis','Mammalian_meat']
extra_1 = ["Conjunctivitis", "Oral_Syndrom", "Cardiovascular_symptoms", "Respiratory_symptoms", "Gastrointestinal_symptoms", "Food_polyallergies"]

Gina = ["GINA_(asthma)_0", "GINA_(asthma)_1", "GINA_(asthma)_2", "GINA_(asthma)_3", "GINA_(asthma)_4", "GINA_(asthma)_5", "GINA_(asthma)_9"]
inconnu = ["Treatment_of_athsma_9", "Treatment_of_rhinitis_9", "General_cofactors_9",
           "Age_of_onsets_9", "ARIA_(rhinitis)_9", "GINA_(asthma)_9", "Treatment_of_atopic_dematitis_9"]
Aria = ["ARIA_(rhinitis)_9", "ARIA_(rhinitis)_0", "ARIA_(rhinitis)_1", "ARIA_(rhinitis)_2", "ARIA_(rhinitis)_3", "ARIA_(rhinitis)_4", "ARIA_(rhinitis)_5"]

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

targets = ["Allergy_Present", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X = V1.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

results = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V1[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results).to_csv("results_V1_Allergie.csv", index=False)


🔍 Target: Allergy_Present | Model: RandomForest
📈 Accuracy: 0.9426
🎯 F1 (0): 0.9400 | F1 (1): 0.9449
📊 Precision: 0.9432 | AUC: 0.981794232370263
📊 Confusion Matrix:
 [[1121    0]
 [   2 1228]]



🔍 Target: Allergy_Present | Model: XGBoost
📈 Accuracy: 0.9400
🎯 F1 (0): 0.9386 | F1 (1): 0.9414
📊 Precision: 0.9414 | AUC: 0.975914184473703
📊 Confusion Matrix:
 [[1121    0]
 [   3 1227]]



🔍 Target: Allergy_Present | Model: LogisticRegression
📈 Accuracy: 0.9409
🎯 F1 (0): 0.9402 | F1 (1): 0.9415
📊 Precision: 0.9437 | AUC: 0.9786801851109536
📊 Confusion Matrix:
 [[1095   26]
 [  87 1143]]



🔍 Target: Allergy_Present | Model: SVM
📈 Accuracy: 0.8532
🎯 F1 (0): 0.8598 | F1 (1): 0.8458
📊 Precision: 0.8676 | AUC: 0.9411421556535414
📊 Confusion Matrix:
 [[1071   50]
 [ 246  984]]



🔍 Target: Respiratory_Allergy | Model: RandomForest
📈 Accuracy: 0.9575
🎯 F1 (0): 0.9612 | F1 (1): 0.9530
📊 Precision: 0.9580 | AUC: 0.9869118890060473
📊 Confusion Matrix:
 [[1288    0]
 [   0 1063]]


...


🔍 Target: Food_Allergy | Model: SVM
📈 Accuracy: 0.8103
🎯 F1 (0): 0.8453 | F1 (1): 0.7545
📊 Precision: 0.8121 | AUC: 0.8829788679361001
📊 Confusion Matrix:
 [[1324  133]
 [ 261  633]]



🔍 Target: Venom_Allergy | Model: RandomForest
📈 Accuracy: 0.9792
🎯 F1 (0): 0.9892 | F1 (1): 0.7224
📊 Precision: 0.9783 | AUC: 0.9542713864306785
📊 Confusion Matrix:
 [[2254    0]
 [   0   97]]



🔍 Target: Venom_Allergy | Model: XGBoost
📈 Accuracy: 0.9821
🎯 F1 (0): 0.9907 | F1 (1): 0.7764
📊 Precision: 0.9819 | AUC: 0.9512725663716814
📊 Confusion Matrix:
 [[2254    0]
 [   0   97]]



🔍 Target: Venom_Allergy | Model: LogisticRegression
📈 Accuracy: 0.9145
🎯 F1 (0): 0.9543 | F1 (1): 0.3347
📊 Precision: 0.9488 | AUC: 0.8160660766961652
📊 Confusion Matrix:
 [[2249    5]
 [  71   26]]



🔍 Target: Venom_Allergy | Model: SVM
📈 Accuracy: 0.8252
🎯 F1 (0): 0.9015 | F1 (1): 0.2187
📊 Precision: 0.9444 | AUC: 0.7936481809242871
📊 Confusion Matrix:
 [[2254    0]
 [  97    0]]


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

V1_sev = V1[V1["Allergy_Present"] == 1]
targets = ["Severe_Allergy"]
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}
X = V1_sev.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_severe = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V1_sev[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_severe.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_severe).to_csv("results_V1_severe.csv", index=False)


🔍 Target: Severe_Allergy | Model: RandomForest
📈 Accuracy: 0.8447
🎯 F1 (0): 0.7878 | F1 (1): 0.8771
📊 Precision: 0.8528 | AUC: 0.9318168604651162
📊 Confusion Matrix:
 [[430   0]
 [  0 800]]



🔍 Target: Severe_Allergy | Model: XGBoost
📈 Accuracy: 0.8431
🎯 F1 (0): 0.7830 | F1 (1): 0.8767
📊 Precision: 0.8484 | AUC: 0.9257848837209302
📊 Confusion Matrix:
 [[430   0]
 [  0 800]]



🔍 Target: Severe_Allergy | Model: LogisticRegression
📈 Accuracy: 0.8195
🎯 F1 (0): 0.7652 | F1 (1): 0.8527
📊 Precision: 0.8359 | AUC: 0.9182558139534883
📊 Confusion Matrix:
 [[362  68]
 [100 700]]



🔍 Target: Severe_Allergy | Model: SVM
📈 Accuracy: 0.6561
🎯 F1 (0): 0.5754 | F1 (1): 0.7102
📊 Precision: 0.6889 | AUC: 0.7212790697674418
📊 Confusion Matrix:
 [[121 309]
 [ 37 763]]


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

# Données respiratoires
V1_res = V1[V1["Respiratory_Allergy"] == 1]

targets = ["Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X = V1_res.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

results_res = []
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Boucle principale
for target in targets:
    y = V1_res[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        print(f"\n🔍 Target: {target} | Model: {model_name}")

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # Application de SMOTE sur les données d'entraînement
            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        # Entraînement final sur tout X (sans SMOTE ici, car prédiction globale)
        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_res.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_res).to_csv("results_V1_respiratoire.csv", index=False)


🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: RandomForest
📈 Accuracy: 0.7413
🎯 F1 (0): 0.7659 | F1 (1): 0.7081
📊 Precision: 0.7456 | AUC: 0.8438125216057385
📊 Confusion Matrix:
 [[579   0]
 [  0 484]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: XGBoost
📈 Accuracy: 0.7516
🎯 F1 (0): 0.7695 | F1 (1): 0.7289
📊 Precision: 0.7556 | AUC: 0.8460580715952443
📊 Confusion Matrix:
 [[579   0]
 [  0 484]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: LogisticRegression
📈 Accuracy: 0.7498
🎯 F1 (0): 0.7793 | F1 (1): 0.7098
📊 Precision: 0.7522 | AUC: 0.8320215949356149
📊 Confusion Matrix:
 [[509  70]
 [152 332]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb | Model: SVM
📈 Accuracy: 0.7140
🎯 F1 (0): 0.7395 | F1 (1): 0.6800
📊 Precision: 0.7166 | AUC: 0.7858488585997012
📊 Confusion Matrix:
 [[475 104]
 [183 301]]



🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Tree | Model: RandomForest
📈 Accuracy: 0.8984
🎯 F1 (0): 0.7999 | F1 (1): 0.9317
📊 Precision: 0.8980 | AUC: 0.9353242543907101
📊 Confusion Matrix:
 [[279   0]
 [  0 784]]


...


🔍 Target: Type_of_Respiratory_Allergy_IGE_Pollen_Gram | Model: SVM
📈 Accuracy: 0.7602
🎯 F1 (0): 0.7050 | F1 (1): 0.7974
📊 Precision: 0.7852 | AUC: 0.8563112782994328
📊 Confusion Matrix:
 [[291  88]
 [136 548]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: RandomForest
📈 Accuracy: 0.9906
🎯 F1 (0): 0.9905 | F1 (1): 0.9907
📊 Precision: 0.9907 | AUC: 0.9991129241129242
📊 Confusion Matrix:
 [[520   0]
 [  0 543]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: XGBoost
📈 Accuracy: 0.9887
🎯 F1 (0): 0.9887 | F1 (1): 0.9887
📊 Precision: 0.9890 | AUC: 0.9993945868945868
📊 Confusion Matrix:
 [[520   0]
 [  0 543]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: LogisticRegression
📈 Accuracy: 0.9877
🎯 F1 (0): 0.9877 | F1 (1): 0.9878
📊 Precision: 0.9881 | AUC: 0.9996082621082621
📊 Confusion Matrix:
 [[519   1]
 [  4 539]]



🔍 Target: Type_of_Respiratory_Allergy_GINA | Model: SVM
📈 Accuracy: 0.6697
🎯 F1 (0): 0.6762 | F1 (1): 0.6614
📊 Precision: 0.6733 | AUC: 0.7325848225848225
📊 Confusion Matrix:
 [[387 133]
 [173 370]]


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

V1_food = V1[V1["Food_Allergy"] == 1]
targets = ["Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X=V1_food.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_food = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V1_food[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_food.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_food).to_csv("results_V1_food.csv", index=False)


🔍 Target: Type_of_Food_Allergy_Aromatics | Model: RandomForest
📈 Accuracy: 0.9776
🎯 F1 (0): 0.9887 | F1 (1): 0.0667
📊 Precision: 0.9662 | AUC: 0.8009273772204806
📊 Confusion Matrix:
 [[878   0]
 [  0  16]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: XGBoost
📈 Accuracy: 0.9754
🎯 F1 (0): 0.9875 | F1 (1): 0.1900
📊 Precision: 0.9727 | AUC: 0.7526907001044931
📊 Confusion Matrix:
 [[878   0]
 [  0  16]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: LogisticRegression
📈 Accuracy: 0.9351
🎯 F1 (0): 0.9662 | F1 (1): 0.0832
📊 Precision: 0.9684 | AUC: 0.7120885579937305
📊 Confusion Matrix:
 [[878   0]
 [ 15   1]]



🔍 Target: Type_of_Food_Allergy_Aromatics | Model: SVM
📈 Accuracy: 0.7998
🎯 F1 (0): 0.8872 | F1 (1): 0.0868
📊 Precision: 0.9737 | AUC: 0.7221068443051202
📊 Confusion Matrix:
 [[878   0]
 [ 16   0]]



🔍 Target: Type_of_Food_Allergy_Cereals_&_Seeds | Model: RandomForest
📈 Accuracy: 0.9608
🎯 F1 (0): 0.9800 | F1 (1): 0.0000
📊 Precision: 0.9403 | AUC: 0.7197986278178738
📊 Confusion Matrix:
 [[867   0]
 [  0  27]]


...


🔍 Target: Type_of_Food_Allergy_TPO | Model: SVM
📈 Accuracy: 0.8064
🎯 F1 (0): 0.8873 | F1 (1): 0.2910
📊 Precision: 0.9433 | AUC: 0.8249019607843138
📊 Confusion Matrix:
 [[848   0]
 [ 46   0]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: RandomForest
📈 Accuracy: 0.9172
🎯 F1 (0): 0.9560 | F1 (1): 0.2820
📊 Precision: 0.8992 | AUC: 0.8458188153310104
📊 Confusion Matrix:
 [[820   0]
 [  0  74]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: XGBoost
📈 Accuracy: 0.9183
🎯 F1 (0): 0.9560 | F1 (1): 0.4085
📊 Precision: 0.9070 | AUC: 0.8562282229965158
📊 Confusion Matrix:
 [[820   0]
 [  0  74]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: LogisticRegression
📈 Accuracy: 0.8681
🎯 F1 (0): 0.9271 | F1 (1): 0.2944
📊 Precision: 0.8830 | AUC: 0.7658101045296167
📊 Confusion Matrix:
 [[813   7]
 [ 60  14]]



🔍 Target: Type_of_Food_Allergy_Tree_Nuts | Model: SVM
📈 Accuracy: 0.7002
🎯 F1 (0): 0.8106 | F1 (1): 0.2641
📊 Precision: 0.8922 | AUC: 0.7194033101045296
📊 Confusion Matrix:
 [[820   0]
 [ 74   0]]


In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score,
    precision_score, confusion_matrix, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE
import plotly.graph_objects as go

V1_venom = V1[V1["Venom_Allergy"] == 1]
targets = ["Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom"]

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

X=V1_venom.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]
results_venom = []

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for target in targets:
    y = V1_venom[target]

    for model_name, base_model in models.items():
        f1_class0_scores, f1_class1_scores = [], []
        precision_scores, acc_scores, recall_scores, auc_scores = [], [], [], []

        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            smote = SMOTE(random_state=42)
            X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            base_model.fit(X_train_res, y_train_res)
            y_pred = base_model.predict(X_test)

            acc_scores.append(accuracy_score(y_test, y_pred))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
            precision_scores.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
            f1_class0_scores.append(f1_score(y_test, y_pred, pos_label=0, zero_division=0))
            f1_class1_scores.append(f1_score(y_test, y_pred, pos_label=1, zero_division=0))

            if hasattr(base_model, "predict_proba"):
                y_proba = base_model.predict_proba(X_test)[:, 1]
                auc_scores.append(roc_auc_score(y_test, y_proba))

        base_model.fit(X, y)
        y_pred_full = base_model.predict(X)
        y_proba_full = base_model.predict_proba(X)[:, 1] if hasattr(base_model, "predict_proba") else None
        matrix = confusion_matrix(y, y_pred_full)

        print(f"\n🔍 Target: {target} | Model: {model_name}")
        print(f"📈 Accuracy: {np.mean(acc_scores):.4f}")
        print(f"🎯 F1 (0): {np.mean(f1_class0_scores):.4f} | F1 (1): {np.mean(f1_class1_scores):.4f}")
        print(f"📊 Precision: {np.mean(precision_scores):.4f} | AUC: {np.mean(auc_scores) if auc_scores else 'N/A'}")
        print("📊 Confusion Matrix:\n", matrix)

        if y_proba_full is not None:
            fpr, tpr, _ = roc_curve(y, y_proba_full)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f"{model_name} (AUC={np.mean(auc_scores):.2f})"))
            fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
            fig.update_layout(
                title=f"ROC Curve - {target} - {model_name}",
                xaxis_title="False Positive Rate",
                yaxis_title="True Positive Rate",
                width=700, height=500
            )
            fig.show()

        results_venom.append({
            "Target": target,
            "Model": model_name,
            "F1_Class_0": np.mean(f1_class0_scores),
            "F1_Class_1": np.mean(f1_class1_scores),
            "Precision": np.mean(precision_scores),
            "Accuracy": np.mean(acc_scores),
            "Recall": np.mean(recall_scores),
            "AUC_ROC": np.mean(auc_scores) if auc_scores else np.nan
        })

pd.DataFrame(results_venom).to_csv("results_V1_venom.csv", index=False)


🔍 Target: Type_of_Venom_Allergy_ATCD_Venom | Model: RandomForest
📈 Accuracy: 0.8944
🎯 F1 (0): 0.9354 | F1 (1): 0.6500
📊 Precision: 0.9175 | AUC: 0.9125
📊 Confusion Matrix:
 [[81  0]
 [ 0 16]]



🔍 Target: Type_of_Venom_Allergy_ATCD_Venom | Model: XGBoost
📈 Accuracy: 0.8744
🎯 F1 (0): 0.9212 | F1 (1): 0.6200
📊 Precision: 0.9061 | AUC: 0.90625
📊 Confusion Matrix:
 [[81  0]
 [ 0 16]]



🔍 Target: Type_of_Venom_Allergy_ATCD_Venom | Model: LogisticRegression
📈 Accuracy: 0.8244
🎯 F1 (0): 0.8919 | F1 (1): 0.4867
📊 Precision: 0.8507 | AUC: 0.8638888888888889
📊 Confusion Matrix:
 [[79  2]
 [ 3 13]]



🔍 Target: Type_of_Venom_Allergy_ATCD_Venom | Model: SVM
📈 Accuracy: 0.6822
🎯 F1 (0): 0.7608 | F1 (1): 0.4655
📊 Precision: 0.8720 | AUC: 0.9027777777777779
📊 Confusion Matrix:
 [[81  0]
 [16  0]]



🔍 Target: Type_of_Venom_Allergy_IGE_Venom | Model: RandomForest
📈 Accuracy: 0.9900
🎯 F1 (0): 0.9000 | F1 (1): 0.9947
📊 Precision: 0.9810 | AUC: 1.0
📊 Confusion Matrix:
 [[10  0]
 [ 0 87]]



🔍 Target: Type_of_Venom_Allergy_IGE_Venom | Model: XGBoost
📈 Accuracy: 1.0000
🎯 F1 (0): 1.0000 | F1 (1): 1.0000
📊 Precision: 1.0000 | AUC: 1.0
📊 Confusion Matrix:
 [[10  0]
 [ 0 87]]



🔍 Target: Type_of_Venom_Allergy_IGE_Venom | Model: LogisticRegression
📈 Accuracy: 0.8622
🎯 F1 (0): 0.3833 | F1 (1): 0.9198
📊 Precision: 0.8786 | AUC: 0.9041666666666666
📊 Confusion Matrix:
 [[10  0]
 [ 0 87]]



🔍 Target: Type_of_Venom_Allergy_IGE_Venom | Model: SVM
📈 Accuracy: 0.6756
🎯 F1 (0): 0.3086 | F1 (1): 0.7755
📊 Precision: 0.8740 | AUC: 0.8458333333333332
📊 Confusion Matrix:
 [[ 0 10]
 [ 0 87]]


#Ne lancer pas cette partie, c pour la recherche des hyperparametres

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from xgboost import XGBClassifier

# Grille des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'gamma': [0, 1, 5]
}

# Données et targets
X = V1.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X = X.iloc[:, 1:]

targets = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Food_Allergy_Other",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",
]

all_results = []

for target in targets:
    print(f"\n🔍 Recherche d'hyperparamètres pour le target: {target}")

    y = V1[target]
    filtered_X = X.copy()
    filtered_y = y.copy()

    if target in ["Severe_Allergy", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]:
        mask = V1["Allergy_Present"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]
    elif target.startswith("Type_of_Respiratory_Allergy_"):
        mask = V1["Respiratory_Allergy"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]
    elif target.startswith("Type_of_Food_Allergy_"):
        mask = V1["Food_Allergy"] == 1
        filtered_X = filtered_X[mask]
        filtered_y = filtered_y[mask]

    if len(filtered_y.unique()) < 2 or len(filtered_y) < 10:
        print(f" Pas assez de données pour {target}, on passe.")
        continue

    model = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='f1',
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(filtered_X, filtered_y)


    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    best_params['Target'] = target
    best_params['Best_F1_Score'] = best_score

    all_results.append(best_params)

df_results = pd.DataFrame(all_results)
df_results = df_results[['Target'] + [col for col in df_results.columns if col != 'Target']]  # Remettre Target en premier
df_results.to_csv("xgboost_best_hyperparameters_V1.csv", index=False)


🔍 Recherche d'hyperparamètres pour le target: Allergy_Present
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Respiratory_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Food_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Venom_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Severe_Allergy
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Food_Allergy_Other
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Respiratory_Allergy_IGE_Pollen_Herb
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

🔍 Recherche d'hyperparamètres pour le target: Type_of_Respiratory_Allergy

#TOP Features

In [20]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import plotly.graph_objects as go

targets = [
    "Allergy_Present", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy",
    "Severe_Allergy", "Type_of_Food_Allergy_Other", "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree", "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach", "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA", "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram", "Type_of_Respiratory_Allergy_GINA",
    "Type_of_Food_Allergy_Aromatics", "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg", "Type_of_Food_Allergy_Fish", "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk", "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes", "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish", "Type_of_Food_Allergy_TPO", "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom", "Type_of_Venom_Allergy_IGE_Venom"
]

inconnu = ["Treatment_of_athsma_9", "Treatment_of_rhinitis_9", "General_cofactors_9",
           "Age_of_onsets_9", "ARIA_(rhinitis)_9", "GINA_(asthma)_9", "Treatment_of_atopic_dematitis_9"]

X = V1.copy()
X.drop(target_1, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X.drop(inconnu, axis=1, inplace=True)
X = X.iloc[:, 1:]

def plot_top_features(model, X_sub, y_sub, target):
    if len(np.unique(y_sub)) < 2:
        print(f"⚠️ Target '{target}' contient une seule classe ({np.unique(y_sub)}). Skip.")
        return

    model.fit(X_sub, y_sub)
    importances = model.feature_importances_
    top_indices = np.argsort(importances)[::-1][:10]
    features = X_sub.columns[top_indices]
    scores = importances[top_indices]

    fig = go.Figure(go.Bar(
        x=scores[::-1],
        y=features[::-1],
        orientation='h',
        text=[f"{s:.3f}" for s in scores[::-1]],
        textposition='outside'
    ))
    fig.update_layout(
        title=f"Top 10 Features pour la cible '{target}' (XGBoost)",
        xaxis_title="Importance",
        yaxis_title="Feature",
        width=800, height=500
    )
    fig.show()

for target in targets:
    X_sub = X.copy()
    y_sub = V1[target]
    model = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)
    plot_top_features(model, X_sub, y_sub, target)